# KNN: K-Nearest Neighbors
In the challenges below, we will develop our own version of the KNN algorithm using nothing but python and numpy.

### Run this cell before moving on
Yay, dependencies!

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.read_csv('auto-mpg.csv')
data = df.values

print(df.head())
print(df.describe())

    mpg  cylinders  displacement  horsepower  weight  acceleration
0  18.0          8         307.0       130.0    3504          12.0
1  15.0          8         350.0       165.0    3693          11.5
2  18.0          8         318.0       150.0    3436          11.0
3  16.0          8         304.0       150.0    3433          12.0
4  17.0          8         302.0       140.0    3449          10.5
              mpg   cylinders  displacement  horsepower       weight  \
count  406.000000  406.000000    406.000000  401.000000   406.000000   
mean    23.504433    5.475369    194.779557  104.937656  2979.413793   
std      7.738736    1.712160    104.922458   38.828773   847.004328   
min      9.000000    3.000000     68.000000   46.000000  1613.000000   
25%     17.500000    4.000000    105.000000   75.000000  2226.500000   
50%     23.000000    4.000000    151.000000   95.000000  2822.500000   
75%     29.000000    8.000000    302.000000  130.000000  3618.250000   
max     46.600000    8

# First, we must first prepare our data
After running the top cell, the __data__ array contains a data set of mpg values ( __y__ labels) in col 0, while the remaining columns contain the __X__ features

We need to split this data into __training data__, which we will use to train our model, and __test data__, which we will use to determine how effictive our model is at making predictions.

A good rule of thumb is to divide your data into 80% training data and 20% test data, although this may differ based on many factors including the size of your data set. Here, we will use the standard 80/20 split.

We cannot simply designate the top 80% of rows as our training data, because this could introduce bias - e.g. what if the data is presented to us ordered by the date it was collected? So, we need to randomly shuffle the observations/rows of our data set before deviding it. We would also like to be able to reproduce the results of our experiments - i.e. we want to shuffle our rows in a random yet reproducible manner.

## Shuffle data

1. Seed your random number generator using np.random.seed()
2. Use np.random.shuffle(arr) to shuffle the data array in-place

## Split data
1. Import the math module
2. Split the data into a 80% training and 20% test data. You may use the *len* and *math.floor()* function to calculate your slice point. save each portion as __train__  and  __test__
3. Now use column slicing to split the __train__ set into __X_train__ and __y_train__, and the __test__ set into __X_test__ and __y_test__ ex: arr[:,colnum]

*Before moving on, make sure you know what data the variables __X_train, y_train, X_test,__ and __y_test__ contain*

## K-Nearest-Neighbors
The K-Nearest-Neighbors algorithm is a supervised learning algorithm that can be used for classification problems or regression problems. (Tomorrow we will discuss the difference between regression and classification). Here we will be using KNN as a regressor, not a classifier.

Regression algorithms try to predict the value of a __label__ based on the values of a number of __features__. In this case, we will try to predict the miles-per-gallon of a car based on the cars displacement, horsepower, weight, acceleration, and number of cylanders.

KNN solves this problem by considering each observation as a point in a space. For example, let's pretend we are trying to predict the mpg of a car based only on its horsepower and its weight. And suppose we have only three observations in our training data:

In [17]:
# Below we have our example training data.
# Each row represents a car model.
# The first value in the row is the horsepower of the car.
# The second value in the row is the weight of the car.
print(data[:3, 2:5:2])

[[  307.  3504.]
 [  350.  3693.]
 [  318.  3436.]]


We can consider the first row, [307, 3504], to be a point in a 2-dimensional space. In this space, the first coordinate represents horsepower and the second coordinate represents weight.

Let's assume we know the mpg for each of the three observations above.
I'll demonstrate the k-nearest-neighbors algorithm using 1 nearest neighbor:

Suppose my friend Shane buys a new car with horsepower == 310 and weight == 3490. If I look at all of the observations in my training data, I see that the closest observation to my friends car is car_one with horsepower == 307 and weight == 3504. Since this observation is more similar to Shane's car than either of the other observations, I will predict that Shane's car has the same mpg as car_one.

If this were a 2-nearest-neighbors algorithm, my prediction for the mpg of Shane's car would be the average of the mpg of the closest two observations to Shane's car.

To start our own implimentation of KNN, we will start by writing a function to find the distance a point is from each point in our training data set.

### get_distances

Define a function __get_distances__ that accepts a  __X__  array (either __X_train__ or __X_test__) and an array of floating point numbers, called __point__.

__point__ will be the same length as all of the individual observations in __X__. Intuitively, __point__ represents an observation that isn't present in our array of observations, __X__.

__get_distances__ should output a one dimensional array. The length of the output should be the number of observations in __X__ (e.g. the entry at the 5th index of the output array should represent the [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between the 5th observation in __X__ and the __point__). If you get stuck, you can follow the step by step guide below. Brownie points for not using the following instructions:

(We will use get_distances([[1,2,3], [4,5,6], [7,8,9]], [1,5,9]) as an example)
1. Use numpy broadcasting to compute an array of differences of the values in each row of __X__ from __point__ and save it in a variable __diffs__ (__diffs__ should be [[0,-3,-6], [3,0,-3], [6, 3, 0]])
3. square each value in every row of __diffs__ and save it in a variable __diffs_sq__ (__diffs_sq__ should be [[0, 9, 36], [9, 0, 9], [36, 9, 0]])
4. Use *np.sum(X,axis=1)* to add the multiple squared distances together (result should be [45, 18, 45])
4. Finally, use *np.sqrt(arr)* to compute the obtain the distances from __point__ for each row, and save this array as __distances__ (should be [6.7082, 4.2426, 6.7082])
5. Return __distances__
6. *__Extension__* give a default parameter to __get_distances__ and assign it 'e' for euclidain, if 'm' is passed, compute the [Manhattan distance](https://en.wiktionary.org/wiki/Manhattan_distance) instead

### find_neighbors
1. Define a function __find_neighbors__ that accepts three parameters: __dists__ an array of distances (i.e. the output of __get_distances__), __y__ an array of labels, and __k__ a variable number of neighbors
2. Find the k lowest items in the __dists__ array, and  save their indices in a variable __indices__
3. Using __indices__, return an array of the y values at those indices

### avg_neighbors
1. Define a function __avg_neighbors__ that recieves an array of values __neighbors__ (i.e. the output of __find_neighbors__)
2. __avg_neighbors__ should return the mean value of all the __neighbors__

### knn_predict
1. define a function __knn_predict__ that will receive an array __X_train__, an array __y_train__, an array __X_test__, and a value __k__
2. __knn_predict__ should use the functions __get_dists__, __find_neighbors__, and __avg_neighbors__ to predict labels for each observation in __X_test__ using __k__ neighbors. It should return an array of predicted labels.

## Let's learn the best value of k!

Now that we have __knn_predict__ working, we can produce a number of different models to make predictions about the mpg of a car based on its other features (i.e. we can make a 1-nearest-neighbor model, a 2-nearest-neighbors model, a 3-nearest-neighbors model, etc.)

We'd like to teach a machine to use an optimal number of neighbors. To do this, we need a way to for our machine to assess how bad a model is. We'd like to ask the question, "which of our possible models is the least bad?".

A function that assesses how good or bad a model is at making predictions is called a __cost function__ or a __loss function__.  There are many cost functions we could pick from, depending on the type of model and type of data we have. For our purposes, we will stick to the __Mean Squared Error__, or __mse__. To calculate a model's __mse__, we use a model to predict the labels for each of the observations in our test data set. We then calculate the square of the difference between our prediction and the actual value of the label. This is the squared error for a single observation. The __mse__ is the mean of the squared errors for each observation in our test data set.

1. Define a function __mse__ that receives an array __predicted__ and an array __actual__
2. __mse__ should return sum of the squared differences, divided by the total number of items

### test_knn
1. define a function __test_knn__ that receives __X_train__, __y_train__, __X_test__, and __y_test__  arrays and a __max_neighbors__ value
2. __test_knn__ should compute the mse of the knn model for all values of K, and then return the value for k that produced the lowest mse value.
3. Test your model on the training set with the same k and compare that to the result to the test set, which is higher? why?

# Extension Questions
1. What is the time complexity and space complexity of knn_predict (where we take input size to be the number of observations in the training data, and all else remains constant)?
2. Suppose, in some hypothetical world, the number of cylinders in an engine was highly correlated to the mpg of a car, while the weight of a car (in kg) had very low correlation to mpg. Why will our measure of distance obscure this correlation?
3. What are some approaches to fixing the problem described in 2? Try looking into data scaling to improve our model.
4. What will happen to our predictions from __knn_predict__ as k gets close to the size of our training set?
5. Is overfitting more likely to be a problem for large or small values of k?
6. How could we use KNN for classification problems instead of regression problems?